In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt

import matplotlib.dates as mdates

### Get data from tencent news

In [ ]:

url_shanghai = "https://i.snssdk.com/forum/ncov_data/?data_type=%5B1%2C2%5D&city_code=%5B%22310000%22%5D&src_type=local&recent_days=150"
url_beijing = "https://i.snssdk.com/forum/ncov_data/?data_type=%5B1%2C2%5D&city_code=%5B%22110000%22%5D&src_type=local&recent_days=150"
url_shenzhen = "https://i.snssdk.com/forum/ncov_data/?data_type=%5B1%5D&city_code=%5B%22440300%22%5D&src_type=local&recent_days=150"
url_changchun = "https://i.snssdk.com/forum/ncov_data/?data_type=%5B1%5D&city_code=%5B%22220100%22%5D&src_type=local&recent_days=150"

js_list = []
# js_list.append(json.loads(requests.get(url_guangzhou).json()['ncov_city_data']['440100']))
js_list.append(json.loads(requests.get(url_shenzhen).json()['ncov_city_data']['440300']))
js_list.append(json.loads(requests.get(url_beijing).json()['ncov_city_data']['110000']))
js_list.append(json.loads(requests.get(url_shanghai).json()['ncov_city_data']['310000']))
# js_list.append(json.loads(requests.get(url_xian).json()['ncov_city_data']['610100']))
js_list.append(json.loads(requests.get(url_changchun).json()['ncov_city_data']['220100']))
# js_list.append(json.loads(requests.get(url_yanbian).json()['ncov_city_data']['222400']))





* confirmedNum: 累计确诊人数
* curesNum：累计治愈人数
* deathsNum: 累计死亡人数
* asymptomaticNum: 现有疑似病例


In [ ]:
df_list = []
for js in js_list:
    df = pd.json_normalize(js,record_path='series', meta='cityName')
    df = df.drop(['curesRatio','deathsRatio','confirmedIncr','curesIncr','deathsIncr', 'asymptomaticIncr','mainlandConfirmedIncr','treatingNum','mainlandTreatingNum','asymptomaticNum','deathsNum','cityName'],axis=1)
    for i in range(95):
        df.iloc[i,1] = df.iloc[i,1] - df.iloc[94,1]
        df.iloc[i,2] = df.iloc[i,2] - df.iloc[94,2]
    df_list.append(df)
    print(df.isna().sum())
df_list[3].drop('nonConfirmedIncrDays',axis=1,inplace=True)
df_list[0]

In [ ]:
for df in df_list:
    df = df.sort_values(by='date')
    df['date'] = pd.to_datetime(df['date']) 
    plt.figure(figsize=(16,9),dpi=100)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.plot(df['date'] ,df['confirmedNum'])
    plt.show()


### 深圳，从2月14日开始选取42天数据

In [ ]:
outputpath = 'shenzhen_data.csv'
df = df_list[0].iloc[-42:]
for i in range(42):
    df.iloc[i,1] = df.iloc[i,1] - df.iloc[41,1]
    df.iloc[i,2] = df.iloc[i,2] - df.iloc[41,2]
df.to_csv(outputpath,sep=',',index=False,header=True)

### 北京，选取最后42天数据

In [ ]:
outputpath = 'beijing_data.csv'
df = df_list[1].iloc[0:42]
for i in range(42):
    df.iloc[i,1] = df.iloc[i,1] - df.iloc[41,1]
    df.iloc[i,2] = df.iloc[i,2] - df.iloc[41,2]
df_list.append(df)
df.to_csv(outputpath,sep=',',index=False,header=True)

### 上海，从3月27日选取42天数据

In [ ]:
outputpath = 'shanghai_data.csv'
df = df_list[2].iloc[12:54]
for i in range(42):
    df.iloc[i,1] = df.iloc[i,1] - df.iloc[41,1]
    df.iloc[i,2] = df.iloc[i,2] - df.iloc[41,2]
df_list.append(df)
df.to_csv(outputpath,sep=',',index=False,header=True)

### 长春，从3月11日开始选取42天数据

In [ ]:
outputpath = 'changchun_data.csv'
df = df_list[3].iloc[28:70]
for i in range(42):
    df.iloc[i,1] = df.iloc[i,1] - df.iloc[41,1]
    df.iloc[i,2] = df.iloc[i,2] - df.iloc[41,2]
df_list.append(df)
df.to_csv(outputpath,sep=',',index=False,header=True)